# 使用 Unsloth 对 DeepSeek-R1-Distill-Qwen-1.5B 模型进行 LoRA 微调

本 Notebook 展示了如何使用 `unsloth` 库对 `deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B` 模型进行高效的 QLoRA (Low-Rank Adaptation) 微调。

整个流程包括：
1.  环境准备与库导入
2.  加载预训练模型和分词器 (Tokenizer)。
3.  在微调前，对模型进行简单的推理测试。
4.  下载和格式化训练数据集
5.  使用 `unsloth` 的 `FastLanguageModel` 来为模型添加 LoRA 适配器。
6.  配置 `SFTTrainer` 监督微调训练配置。
7.  启动训练，并观察 Loss 变化情况
8.  保存微调后的模型
9.  测试训练后的生成结果

### 1. 环境准备与库导入

首先，我们需要安装并导入所有必要的库。`transformers` 用于加载模型和分词器，`unsloth` 用于高效微调，`trl` 提供了 `SFTTrainer`，而 `datasets` 用于处理数据。

**注意**: 在运行此 Notebook 之前，请确保已安装所有依赖包：
```bash
pip install -r requirements.txt
```

In [1]:
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig, DataCollatorForSeq2Seq
from datasets import Dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


### 2. 加载预训练模型和分词器 (Tokenizer)

In [2]:
# 定义模型和一些基本参数
max_seq_length = 8192
dtype = None # None 表示自动选择 (Float16 a T4, V100, BFloat16 a Ampere)
load_in_4bit = True # 使用 4bit 量化加载

# 这是您的模型标识符，请替换为您正在使用的模型
# 例如："qwen-1.5b_lora_model"
# model_name = "qwen-1.5b_lora_model" 
# model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B" 
model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit" 

# 这一步会返回一个经过 Unsloth 优化的模型和一个分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.8.5: Fast Qwen2 patching. Transformers: 4.55.2.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 2. Max memory: 11.63 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


### 3. 微调前推理测试

在对模型进行任何修改之前，我们先用它来生成一段文本，看看原始模型的表现如何。这可以作为我们微调效果的基准参考。

In [3]:
# 模型推理的 Prompt 模板
inference_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>{}
"""

In [4]:
FastLanguageModel.for_inference(model)

question = "男，28岁，程序员，最近一周每天工作到半夜，感觉头晕、脖子疼，有时候还恶心。"

inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")
attention_mask = inputs.input_ids.ne(tokenizer.pad_token_id).long().to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

In [5]:
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [6]:
print(response[0].split("### Response:")[1])


<think>
首先，我需要分析用户提供的信息。用户是一名28岁的男性，程序员，最近一周每天工作到半夜，表现出头晕、脖子疼、恶心等症状。

接下来，我需要考虑用户的健康风险。头晕、脖子疼和恶心这些症状可能提示有中枢神经系统疾病，如中风、脑膜炎或脑梗死。这些疾病通常伴随的伴随性脑力损伤，可能需要及时就医。

然后，我需要评估是否有其他潜在的健康风险。例如，长时间工作可能引发脑力损伤，但这种情况通常是正常的， unless有严重并发症。

最后，我需要考虑用户的年龄和工作模式。28岁是个中年人，可能有中风的危险。程序员的工作环境可能不适合他，但这是正常的， unless他长期使用电子设备。

综合以上分析，建议用户立即就医，进行脑力损伤评估，以确保是否需要立即处理。如果症状持续或加重，应立即寻求专业医疗帮助。
</think>

请立即就医。头晕、脖子疼和恶心可能提示有中枢神经系统疾病，如中风、脑膜炎或脑梗死。这些疾病通常伴随伴随性脑力损伤。请立即联系医疗机构进行评估。


---

### 4. 下载和格式化训练数据集


医学推理数据集：https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT/viewer/zh

![dataset](images/dataset.png)

In [7]:
# 模型训练的 Prompt 模板
train_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>
{}
</think>
{}
"""

In [8]:
EOS_TOKEN = tokenizer.eos_token # 添加 EOS Token

def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        # 将 EOS Token 添加到样本最后
        text = train_prompt.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "zh", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/20171 [00:00<?, ? examples/s]

In [9]:
dataset[0]["text"]

'以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。\n请撰写一份恰当的回复，以完成该请求。\n在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。\n\n\n### Instruction:\n你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。\n请回答以下医学问题。\n\n### Question:\n根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？\n\n### Response:\n<think>\n这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？\n\n再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。\n\n看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。\n\n不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。\n\n

In [10]:
from IPython.display import display, Markdown

display(Markdown(dataset[0]["text"]))

以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？

### Response:
<think>
这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。

用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。

但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。

等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。

仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。

回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？

再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。

看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。

不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。

哦，该不会是夏季湿热，导致湿毒入侵，孩子的体质不能御，其病情发展成这样的感染？综合分析后我觉得‘蝼蛄疖’这个病名真是相当符合。
</think>
从中医的角度来看，你所描述的症状符合“蝼蛄疖”的病症。这种病症通常发生在头皮，表现为多处结节，溃破流脓，形成空洞，患处皮肤增厚且长期不愈合。湿热较重的夏季更容易导致这种病症的发展，特别是在免疫力较弱的儿童身上。建议结合中医的清热解毒、祛湿消肿的治疗方法进行处理，并配合专业的医疗建议进行详细诊断和治疗。
<｜end▁of▁sentence｜>

### 5. 使用 Unsloth 添加 LoRA 适配器

这是使用 `unsloth` 的核心步骤。我们调用 `FastLanguageModel.get_peft_model`，它会非常高效地为模型注入 LoRA 模块。

- `r`: LoRA 的秩 (rank)，是控制模型复杂度和参数量的关键超参数。
- `target_modules`: 指定要在哪些线性层（如注意力机制的 q, k, v, o 投影层）上应用 LoRA。
- `lora_alpha`: LoRA 的缩放因子，通常设置为 `r` 的两倍或与 `r` 相同。
- `use_gradient_checkpointing`: 一种节省显存的技术，对于训练大模型至关重要。

In [11]:
# 因为 `model` 对象现在是由 Unsloth 创建的，它包含了所有必需的属性
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
      "q_proj",
      "k_proj",
      "v_proj",
      "o_proj",
      "gate_proj",
      "up_proj",
      "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=1432,
    use_rslora=False,
    loftq_config=None,
)
# 检查模型结构，确认 LoRA 适配器已添加
print(model)

Unsloth 2025.8.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536, padding_idx=151654)
        (layers): ModuleList(
          (0): Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
       

### 6. 配置 SFTTrainer

`SFTTrainer` (Supervised Fine-tuning Trainer) 是一个专门用于指令微调的训练器。我们需要配置 `TrainingArguments` 来指定所有的训练参数，如批量大小、学习率、优化器等。

In [12]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 64,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 1432,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/20171 [00:00<?, ? examples/s]

### 7. 开始训练

一切准备就绪后，调用 `trainer.train()` 即可开始微调过程。训练结束后，会返回包含训练统计信息（如训练损失）的对象。

In [13]:
trainer_stats = trainer.train()

# 打印训练统计信息
print(trainer_stats)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 128 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (128 x 2 x 1) = 256
 "-____-"     Trainable parameters = 18,464,768 of 1,795,552,768 (1.03% trained)
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 115 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (115 x 2 x 1) = 230
 "-____-"     Trainable parameters = 18,464,768 of 1,795,552,768 (1.03% trained)
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 103 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs

Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.160900
2,3.172600
3,3.074500
4,3.184300
5,3.138900
6,3.007200
7,2.977900
8,2.947800
9,2.876300
10,2.829400


TrainOutput(global_step=60, training_loss=2.4791844685872397, metrics={'train_runtime': 3599.8436, 'train_samples_per_second': 2.167, 'train_steps_per_second': 0.017, 'total_flos': 7.31800443009792e+16, 'train_loss': 2.4791844685872397})


### 8. 保存微调后的模型（Lora）

训练完成后，您可以再次进行推理，比较微调后的模型与原始模型的差异。如果对结果满意，可以使用 `model.save_pretrained("your_lora_adapter_path")` 来保存训练好的 LoRA 适配器。

In [14]:
model.save_pretrained("qwen-1.5b_lora_model")

In [15]:
tokenizer.save_pretrained("qwen-1.5b_lora_model")

('qwen-1.5b_lora_model/tokenizer_config.json',
 'qwen-1.5b_lora_model/special_tokens_map.json',
 'qwen-1.5b_lora_model/chat_template.jinja',
 'qwen-1.5b_lora_model/tokenizer.json')

In [16]:
# 模型保存方式二选一（要么使用上面的分开保存，要么使用这里的合并 Lora 保存）
# model.save_pretrained_merged("qwen-1.5b_lora_model", tokenizer, save_method="merged_16bit")

### 9. 测试训练后的生成结果

In [17]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

question="一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？", # Question
inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1000,
    use_cache=True,
)

In [18]:
output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(output[0].split("### Response:")[1])


<think>
啊，这位病人已经急性阑尾炎了，而且已经发病了5天。好，他现在有轻微的腹痛，但是还是有发热的。嗯，看来他可能还有些热能。不过，重要的是，他现在还发现右下腹有压痛的包块。啊，这可能是个问题，尤其是如果他有热能，压痛包块可能会引起不适。

首先，我得想到，阑尾炎患者应该保持良好的卫生习惯，避免感染。所以，我应该给他换上换季的卫生用品，比如洗面剂和牙膏，以防止感染。

然后，关于腹痛和发热，这可能是由于感染引起的。如果他的疼痛没有消退，而发热却有，可能需要更仔细地评估他的病情。比如，如果他感到发热，但没有明显的发热症状，应该先考虑感染性休克。

此外，如果他有压痛包块，这可能影响他的生命质量。所以，我应该提醒他，如果症状严重，最好立即就医。

哦，对了，为了预防感染，我应该提醒他避免接触高热。因为高热会增加感染的风险。

最后，我得想一下，如果他没有发烧，而只是有轻微的腹痛和发热，那可能他只是有轻微的感染。这时候，他应该继续观察，看症状是否有所改善。

哦，等等，我得再理一下。如果他的症状是轻微的，可能需要稍后再考虑感染的可能性。不过，还是得提醒他避免高热，因为高热会增加感染的风险。

哦，对了，如果他的腹痛和发热在5天内没有改善，就有可能是感染性的休克，尤其是如果是由于感染引起的。所以，这个时候应该立即就医，看是否有感染性休克。

嗯，看来我需要把这些点都列出来，确保回复全面。


回复：
首先，根据病人的情况，他患有急性阑尾炎，已经发病了5天。因此，我们首先应该考虑他的整体健康状况，避免感染。换上换季的卫生用品是必要的，以预防感染。

接下来，检查他的腹痛和发热情况。如果腹痛没有明显减轻，而发热却仍然存在，这可能表明感染性休克的可能。在这种情况下，我们应立即就医，进行初步检查，以确认感染性休克的性质。

此外，如果病人有压痛包块，这可能影响生命质量。因此，我们应立即采取措施，避免接触高热，以防止感染。如果症状没有改善，我们应再次就医，以确保及时处理。

总之，尽管病人有轻微的腹痛和发热，但考虑到他的病情，我们建议立即就医以确认感染性休克，以便进行进一步的治疗。
</think>
根据病人的情况，他患有急性阑尾炎，已经发病了5天。因此，首先应该考虑他的整体健康状况，避免感染。换上换季的卫生用品以预防感染是必要的。

然后，检查他的腹痛和发热情况。如果腹痛

In [19]:
def generate_response(question: str, model, tokenizer, inference_prompt: str, max_new_tokens: int = 1024) -> str:
    """
    使用指定的模型和分词器为给定的医学问题生成响应。

    Args:
        question (str): 需要模型回答的医学问题。
        model: 已加载的 Unsloth/Hugging Face 模型。
        tokenizer: 对应的分词器。
        inference_prompt (str): 用于格式化输入的 f-string 模板。
        max_new_tokens (int, optional): 生成响应的最大 token 数量。默认为 1024。

    Returns:
        str: 模型生成的响应文本，已去除 prompt 部分。
    """
    # 1. 使用模板格式化输入
    prompt = inference_prompt.format(
        question, # 填充问题
        "",       # 留空，让模型生成 CoT 和 Response
    )

    # 2. 将格式化后的 prompt 进行分词，并转移到 GPU
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    # 3. 使用模型生成输出
    # use_cache=True 用于加速解码过程
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=max_new_tokens,
        use_cache=True,
    )
    
    # 4. 将生成的 token 解码为文本
    # skip_special_tokens=True 会移除像 EOS_TOKEN 这样的特殊标记
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # 5. 切分字符串，只返回 "### Response:" 之后的部分
    # 使用 .split() 分割并获取响应内容，.strip() 用于去除可能存在的前后空白字符
    response_part = decoded_output.split("### Response:")
    if len(response_part) > 1:
        return response_part[1].strip()
    else:
        # 如果模型没有生成 "### Response:" 标记，则返回整个生成内容以供调试
        return decoded_output

In [20]:
my_question = "对于一名60岁男性患者，出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液，请问哪一项实验室检查对了解胸水的性质更有帮助？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
嗯，这个60岁的男患者，有右侧胸疼，X线显示肋膈角消失，这让我想到肺结核。肺结核通常会导致胸腔积液，但具体是哪种呢？我得想想。

首先，肺结核患者通常会有胸腔积液，但这是不是单纯的胸腔积液呢？我记得有两类，胸腔积液和肺栓塞。胸腔积液可能跟肺泡病变有关，而肺栓塞则可能是因为肺泡的通透性改变。

然后，我得想想，胸腔积液的性质是什么。我好像记得，胸腔积液可能有水泡和水泡中充血。水泡可能来自于胸膜下部的肺泡病变，而水泡中充血可能是因为肺泡壁细胞和淋巴细胞的融合。

那实验室检查应该考虑哪些呢？胸腔积液的性质主要看水泡的大小和形态，还有水泡中充血的类型。比如，如果水泡很大，可能有水泡、水泡中充血或水泡和水泡中充血都可能。

所以，哪个检查能帮到我们呢？水泡的形态和大小能告诉我，水泡是来自于哪里。水泡中充血的类型也很重要，能进一步了解肺泡的病变情况。

所以，选择水泡形态、大小、形态和水泡中充血的类型，这些都能帮助我们了解胸腔积液的性质。

哦，对了，像水泡、水泡中充血和水泡和水泡中充血这些检查，都能提供关键信息。比如，水泡的形态大，可能说明肺泡病变在胸膜下部，而水泡中充血可能说明肺泡通透性变化。水泡和水泡中充血则可能反映出肺泡的通透性改变。

所以，综合来看，这些检查能帮助我们更准确地了解胸腔积液的性质。
</think>
对于60岁男性患者出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液，实验室检查可以提供关键信息来了解胸腔积液的性质。

胸腔积液的性质通常可以通过以下几种检查来评估：
1. **水泡形态和大小**：水泡的形态和大小可以反映肺泡病变的位置。水泡大且与胸膜下部的肺泡病变相符，通常与肺泡病变相关。
2. **水泡中充血类型**：水泡中充血的类型可以帮助评估肺泡通透性。水泡和水泡中充血可能是由于肺泡通透性改变，而水泡中充血则可能表明肺泡通透性增强。
3. **水泡和水泡中充血的类型**：这也可能反映肺泡通透性变化，可以帮助确定肺泡病变的具体位置。

因此，水泡形态、大小、形态以及水泡中充血的类型这些实验室检查对了解胸腔积液的性质是有帮助的。


In [22]:
my_question = "对于一名 28 岁的男性患者，工作是程序员，常年熬夜，最近突然感觉头晕目眩，甚至有点恶心。请问有可能是什么疾病？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
这个患者是28岁的男性，工作是程序员，通常工作时间很长，熬夜很多天。突然感觉头晕目眩，还有点恶心，这是很让人担心的事情。

首先，头晕目眩和恶心是常见的身体反应，但结合工作长期熬夜，这可能提示他有慢性疾病。

一个常见的解释是心脏问题。长期熬夜，尤其是如果心脏有问题，可能会导致头晕、恶心和心悸。这个患者可能有心脏病，尤其是心力衰竭。

心力衰竭是心脏功能严重受损，导致心脏无法正常工作，从而产生头晕、恶心、心悸等症状。

所以，这个患者可能患有心力衰竭，需要及时处理以减少症状并改善身体状况。
</think>
根据您的描述，这位28岁的男性患者在工作时间长期熬夜后突然出现头晕目眩和恶心等症状，这提示他可能患有心力衰竭。心力衰竭是一种心力不恢复性疾病，通常表现为心率加快、心肌缺血、心悸、头晕目眩等症状。这种疾病可能与长期的熬夜和心脏功能受损有关。

心力衰竭的患者通常需要立即采取措施以控制症状，并评估是否需要进行心脏手术或其他治疗。如果心力衰竭的严重程度和影响持续存在，建议尽快就医进行检查和治疗。

总结来说，这位患者可能患有心力衰竭，需要及时处理以减少症状并改善身体状况。
